In [ ]:
import requests
import pandas as pd

In [ ]:
base_url="http://b2find.eudat.eu/solr/select?echoParams=none&rows=0&wt=json&q=*&facet=false&fq={fq}"

In [ ]:
query = 'extras_TempCoverageBegin:[* TO {end}] AND extras_TempCoverageEnd:[{start} TO *]'


In [ ]:

# fq=query.format(start='1900-01-01T00:00:00Z', end='2021-01-01T00:00:00Z')
min=0
max=320000000000
num_intervals=50
interval=int((max-min)/num_intervals)

In [ ]:
import time
run_start = time.time()

x = []
y = []

for i in range(num_intervals):
    start = int(min + i * interval)
    end = int(min + (i+1) * interval)
    # print(start, end, interval)
    fq = query.format(start=start, end=end)
    url = base_url.format(fq=fq)
    resp = requests.get(url)
    if resp.ok:
        num_found = resp.json()['response']['numFound']
        x.append(start)
        y.append(num_found)
    else:
        print("Failed")
        
run_end = time.time() - run_start
run_end

In [ ]:
df = pd.DataFrame(list(zip(x, y)),
               columns =['start', 'counts'])
df

In [ ]:
df["start"]=pd.to_datetime(df["start"],unit="ms")
df

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df)

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(df.start.loc[10], df.start.loc[20]))

p.line('start', 'counts', source=source)
p.yaxis.axis_label = 'datasets'

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('start', 'counts', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))